## Lab. 12

### Solve the following problem using Genetic Algorithms:


Problem: Weighted N-Queen Problem


You are given an N×N chessboard, and each cell of the board has an associated weight. Your task is to find a valid placement of N queens such that the total weight of the queens is maximized, and no two queens threaten each other.





In the traditional N-Queen Problem, the goal is to place N queens on an N×N chessboard in such a way that no two queens threaten each other. In this variation, we introduce weights to the queens and aim to find a placement that maximizes the total weight of the queens while satisfying the constraint of non-threatening positions.


Constraints:

1. There should be exactly one queen in each row and each column.
2. No two queens should be placed in the same diagonal, i.e., they should not threaten each other.
3. The placement should maximize the total weight of the queens.


Representation:

Use a permutation-based representation. Each permutation represents the column position of the queen for each row. 

For example, if N=4, a valid permutation [2, 4, 1, 3] indicates that the queen in the first row is placed in column 2, the queen in the second row is placed in column 4, and so on.


Genetic Algorithm Steps:

1. *Initialization*: Generate an initial population of permutations randomly.

2. *Fitness Evaluation*: Evaluate the fitness of each permutation by calculating the total weight of the queens while considering the non-threatening positions.

3. *Selection*: Select a subset of permutations from the population based on their fitness, using selection techniques like tournament selection or roulette wheel selection.

4. *Crossover*: Perform crossover (recombination) on the selected permutations to create new offspring permutations.

5. *Mutation*: Introduce random changes (mutations) in the offspring permutations to maintain diversity in the population.

6. *Fitness Evaluation for the new individuals*: Evaluate the fitness of the new population.

7. *Form the new population*: Select the surviving individuals based on scores, with chances direct proportional with their performance.

8. Repeat steps 3-7 for a certain number of generations or until a termination condition is met (e.g., a maximum number of iterations or a satisfactory solution is found).


9. *Termination*: Return the best-performing individual (permutation) found as the solution to the problem.

Note: The fitness function used in this problem should calculate the total weight of the queens based on the positions specified by the permutation. Additionally, the fitness function should penalize solutions that violate the non-threatening constraint by assigning a lower fitness score to such permutations.

In [1]:
import numpy as np

def initialize_population(population_size, N):
    population = []
    for _ in range(population_size):
        permutation = np.random.permutation(N) + 1  
        population.append(permutation)
    return population

def calculate_fitness(population, weights):
    fitness_scores = []
    for individual in population:
        fitness = 0
        penalty = 0
        for row, col in enumerate(individual):
            threatening = False
            for prev_row in range(row):
                if individual[prev_row] == col or abs(individual[prev_row] - col) == row - prev_row:
                    threatening = True
                    break
            if not threatening:
                fitness += weights[row][col - 1] 
            else:
                penalty += 1 
        
        non_threatening_penalty = max(0, len(individual) - len(set(individual)))  
        penalty += non_threatening_penalty
        
        diagonal_positions = [individual[i] + i for i in range(len(individual))]  
        if len(set(diagonal_positions)) != len(individual):
            penalty += len(individual) - len(set(diagonal_positions))
        
        fitness -= penalty 
        
        fitness_scores.append(max(fitness, 0))  
    
    return fitness_scores

def tournament_selection(population, fitness_scores, tournament_size):
    selected_population = []
    for _ in range(len(population)):
        tournament_indices = np.random.choice(len(population), tournament_size, replace=False)
        tournament_fitness = [fitness_scores[i] for i in tournament_indices]
        winner_index = tournament_indices[np.argmax(tournament_fitness)]
        selected_population.append(population[winner_index])
    return selected_population

def perform_crossover(selected_population):
    offspring_population = []
    for i in range(0, len(selected_population), 2):
        parent1 = selected_population[i]
        parent2 = selected_population[i + 1]
        child1, child2 = crossover(parent1, parent2)
        offspring_population.extend([child1, child2])
    return offspring_population

def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))  # Select random crossover point
    child1 = np.concatenate((parent1[:crossover_point], [gene for gene in parent2 if gene not in parent1[:crossover_point]]))
    child2 = np.concatenate((parent2[:crossover_point], [gene for gene in parent1 if gene not in parent2[:crossover_point]]))
    return child1, child2


def mutate(individual, mutation_rate):
    mutated_individual = np.copy(individual)
    for i in range(len(mutated_individual)):
        if np.random.rand() < mutation_rate:
            # Mutate the position of the queen in this row
            mutated_individual[i] = np.random.randint(1, len(mutated_individual) + 1)
    return mutated_individual

def perform_mutation(population, mutation_rate):
    mutated_population = []
    for individual in population:
        mutated_individual = mutate(individual, mutation_rate)
        mutated_individual = mutated_individual
        mutated_population.append(mutated_individual)
    return mutated_population

def fitness_proportional_selection(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    probabilities = [score / total_fitness for score in fitness_scores]
    
    selected_population_indices = np.random.choice(len(population), size=len(population), p=probabilities, replace=True)
    selected_population = [population[idx] for idx in selected_population_indices]
    
    return selected_population


def evolutionary_algorithm(N, population_size, max_generations, mutation_rate, tournament_size, weights):
    population = initialize_population(population_size, N)
    best_individual = None
    best_fitness = float('-inf')

    for generation in range(1, max_generations + 1):
        fitness_scores = calculate_fitness(population, weights)
        
        selected_population = tournament_selection(population, fitness_scores, tournament_size)
        
        offspring_population = perform_crossover(selected_population)
        
        mutated_population = perform_mutation(offspring_population, mutation_rate)
        
        mutated_fitness_scores = calculate_fitness(mutated_population, weights)
        
        new_population = fitness_proportional_selection(mutated_population, mutated_fitness_scores)
        
        max_fitness_index = np.argmax(mutated_fitness_scores)
        if mutated_fitness_scores[max_fitness_index] > best_fitness:
            best_individual = mutated_population[max_fitness_index]
            best_fitness = mutated_fitness_scores[max_fitness_index]
        
        if generation == max_generations:
            break
        
        population = new_population
    
    return best_individual
    
N = 5
population_size = 100 
max_generations = 100 
mutation_rate = 0.1 
tournament_size = 30

weights = np.random.randint(1, 10, size=(N, N))  

best_solution = evolutionary_algorithm(N, population_size, max_generations, mutation_rate, tournament_size, weights)

print("Best solution found:")
print(best_solution)

Best solution found:
[4 2 5 3 1]
